### When running this notebook via the Galaxy portal
You can access your data via the dataset number. Using a Python kernel, you can access dataset number 42 with ``handle = open(get(42), 'r')``.
To save data, write your data to a file, and then call ``put('filename.txt')``. The dataset will then be available in your galaxy history.
<br><br>Note that if you are putting/getting to/from a different history than your default history, you must also provide the history-id.
<br><br>More information including available galaxy-related environment variables can be found at https://github.com/bgruening/docker-jupyter-notebook. This notebook is running in a docker container based on the Docker Jupyter container described in that link.


# Dilepton analysis  (Python) 


This is an example of a simple dilepton analysis, quite similar to the notebook called "Dilepton_analysis_noData.ipynb", but with some differences. The most obvious difference is that we here also include real data. This example also has a slightly more advanced event selection.  

**Notice:** This is *only an example* on how to do this. Feel free to be creative, and to find better and/or more elegant ways of doing the various steps! 

In [1]:
import ROOT as R
import import_ipynb
import setPath
from os import listdir
from os.path import isfile, join
from Input.OpenDataPandaFramework13TeV import *
%jsroot on

Welcome to JupyROOT 6.24/02
importing Jupyter notebook from setPath.ipynb
importing Jupyter notebook from /storage/galaxy/jobs_directory/003/3037/working/jupyter/Input/OpenDataPandaFramework13TeV.ipynb
This library contains handy functions to ease the access and use of the 13TeV ATLAS OpenData release

getBkgCategories()
	 Dumps the name of the various background cataegories available 
	 as well as the number of samples contained in each category.
	 Returns a vector with the name of the categories

getSamplesInCategory(cat)
	 Dumps the name of the samples contained in a given category (cat)
	 Returns dictionary with keys being DSIDs and values physics process name from filename.

getMCCategory()
	 Returns dictionary with keys DSID and values MC category

initialize(indir)
	 Collects all the root files available in a certain directory (indir)



Setting luminosity to 10064 pb^-1

###############################
#### Background categories ####
###############################
Category    

## 1. Reading the dataset

Set the analaysis to run (*1largeRjet1lep*, *1lep1tau*, *3lep*, *exactly2lep*, *GamGam*, *2lep*, *4lep*)

Set the directory where you have downloaded the ATLAS OpenData samples you want to run over

In [2]:
opendatadir = "/storage/shared/data/fys5555/ATLAS_opendata/"
analysis = "2lep"

In [3]:
background = R.TChain("mini")
data = R.TChain("mini")

A list of all the background samples, category and their IDs can be found in **Infofile.txt**. The cross-section, efficiencies etc. needed for scaling are stored in the **Files_<---->**. We read these files and add all the samples to the TChain. We also (for later convenience) make a vector containing the dataset IDs. 

In [4]:
mcfiles = initialize(opendatadir+"/"+analysis+"/MC")
datafiles = initialize(opendatadir+"/"+analysis+"/Data")
allfiles = z = {**mcfiles, **datafiles}
Backgrounds = getBkgCategories()
Backgrounds = ["Diboson", "ttbar", "singleTop", "SUSYSlepSlep"]

WARNING 	 File topX.root not added as sample in Background_samples_13TeV.txt/Signal_samples_13TeV.txt
WARNING 	 File Zjetsincl.root not added as sample in Background_samples_13TeV.txt/Signal_samples_13TeV.txt
WARNING 	 File Diboson.root not added as sample in Background_samples_13TeV.txt/Signal_samples_13TeV.txt
WARNING 	 File Gmumu.root not added as sample in Background_samples_13TeV.txt/Signal_samples_13TeV.txt
WARNING 	 File ZPrimeee.root not added as sample in Background_samples_13TeV.txt/Signal_samples_13TeV.txt
WARNING 	 File dmV_Zll.root not added as sample in Background_samples_13TeV.txt/Signal_samples_13TeV.txt
WARNING 	 File RS_G_ZZ.root not added as sample in Background_samples_13TeV.txt/Signal_samples_13TeV.txt
WARNING 	 File Higgs.root not added as sample in Background_samples_13TeV.txt/Signal_samples_13TeV.txt
WARNING 	 File ZPrimemumu.root not added as sample in Background_samples_13TeV.txt/Signal_samples_13TeV.txt
WARNING 	 File ZPrimett.root not added as sample in Back

In [5]:
MCcat = {}
for cat in allfiles:
    for dsid in allfiles[cat]["dsid"]:
        try:
            MCcat[int(dsid)] = cat
        except:
            continue

In [6]:
dataset_IDs = []
background.Reset()
for b in Backgrounds:
    i = 0
    if not b in mcfiles.keys(): continue
    for mc in mcfiles[b]["files"]:
        if not os.path.isfile(mc): continue
        try:
            dataset_IDs.append(int(mcfiles[b]["dsid"][i]))
            background.Add(mc)
        except:
            print("Could not get DSID for %s. Skipping"%mc)
        i += 1
nen = background.GetEntries()
print("Added %i entries for backgrounds"%(nen))

Could not get DSID for /storage/shared/data/fys5555/ATLAS_opendata//2lep/MC/Diboson.root. Skipping
Could not get DSID for /storage/shared/data/fys5555/ATLAS_opendata//2lep/MC/ttbar.root. Skipping
Could not get DSID for /storage/shared/data/fys5555/ATLAS_opendata//2lep/MC/singleTop.root. Skipping
Could not get DSID for /storage/shared/data/fys5555/ATLAS_opendata//2lep/MC/SUSYSlepSlep.root. Skipping
Added 21867152 entries for backgrounds


In [7]:
data.Reset(); 
for d in datafiles["data"]["files"]:  
    if not os.path.isfile(d): continue
    data.Add(d)
nen = data.GetEntries()
print("Added %i entries for backgrounds"%(nen))

Added 24411580 entries for backgrounds


## 2. Making (a lot of) histograms

Now that we have read our dataset we want to start analyzing the data. To do so we need to put the data into histograms. For reasons that will become clear later in the analysis we must (for each variable) make one histogram per dataset ID. (We have 31 background samples, so if we want to study 10 variables we have to make 310 histograms!) For best dealing with all these histograms we can use dictionaries (Python) or maps (C++). 

In [8]:
hist_mll = {}; hist_lep_pt = {}; hist_met = {}; 

In [9]:
for i in dataset_IDs: 
    hist_mll[i] = R.TH1F() 
    hist_lep_pt[i] = R.TH1F()
    hist_met[i] = R.TH1F()

In [10]:
for i in dataset_IDs: 
    hist_mll[i].SetNameTitle("hist_mll", "Invariant mass"); 
    hist_lep_pt[i].SetNameTitle("hist_lep_pt", "Lepton pT"); 
    hist_met[i].SetNameTitle("hist_met", "Missing ET");
    hist_mll[i].SetBins(20,0,500); 
    hist_lep_pt[i].SetBins(20,0,1000);
    hist_met[i].SetBins(20,0,500); 

For data it is only necessary with one histogram for each variable: 

In [11]:
hist_mll_d = R.TH1F(); 
hist_lep_pt_d = R.TH1F(); 
hist_met_d = R.TH1F(); 

In [12]:
hist_mll_d.SetNameTitle("hist_mll", "Invariant mass"); 
hist_lep_pt_d.SetNameTitle("hist_lep_pt", "Lepton pT"); 
hist_met_d.SetNameTitle("hist_met", "Missing ET");
hist_mll_d.SetBins(20,0,500); 
hist_lep_pt_d.SetBins(20,0,1000);
hist_met_d.SetBins(20,0,500); 

In [13]:
# Retrieve lumi from library
%store -r lumi

no stored variable or alias lumi


### 2.1 Fill the histograms 
We can now loop over all events in our dataset, implement desired cuts, and fill the histograms we created above. In this example we choose only events containing exactly to same flavour leptons with opposite charge (i.e. $e^+e^-$ or $\mu^+\mu^-$). 
Before starting the loop we extract the total number of entries (events) in the TChain. We also make [TLorentzVector](https://root.cern.ch/doc/master/classTLorentzVector.html)s, which are very practical for handling the kinematics of the leptons, e.g. calculating the invariant mass of the two leptons. 

In [14]:
for i in dataset_IDs: 
    hist_mll[i].Reset(); 
    hist_lep_pt[i].Reset(); 
    hist_met[i].Reset();

In [15]:
hist_mll_d.Reset(); 
hist_lep_pt_d.Reset(); 
hist_met_d.Reset(); 

In [16]:
l1 = R.TLorentzVector() 
l2 = R.TLorentzVector() 
dileptons = R.TLorentzVector() 

This is the cell where the analysis is performed. Note that the cell needs to be run twice:

1. with data = 0 to run over MC
2. with data = 1 to run over data

Note that the MC running takes ~5 minutes for 3lep analysis. Much(!!!) more time for e.g. 2lep analysis! Data running is relatively fast for 3lep. 

In [17]:
import pandas as pd

In [20]:
%%time
import time
from math import sin
isData = 0; 

if isData == 1: ds = data 
else: ds = background
    
columns = {"lep_pt1":[],"lep_eta1":[],"lep_phi1":[],"lep_E1":[], "lep_m1":[],
           "lep_pt2":[],"lep_eta2":[],"lep_phi2":[],"lep_E2":[], "lep_m2":[],
           "met":[], "weight": [], "label": []}

i = 0   
for event in ds: 
    
    if i%100000 == 0 and i>0: 
        print("Total events %i/%i"%(i,ds.GetEntries()))
    i += 1 
    ## Data quality cuts: 
    
    #if i > 10000000: break
    
    if(ds.trigM == 0 and ds.trigE == 0):continue  

    ## Event selection:    
    ## Cut #1: Require (exactly) 2 leptons.
    if not ds.lep_n == 2: continue
    ## Cut #2: Require opposite charge.
    if ds.lep_charge[0] == ds.lep_charge[1]: continue
    ## Cut #3: Require same flavour (2 electrons or 2 muons).
    if not ds.lep_type[0] == ds.lep_type[1]: continue
    ## Cut #4: Require a significant amount of missing energy.
    
    #New cuts
    if ds.met_et/1000.0 < 100: continue
    
    ## Require "good leptons": 
    if ds.lep_pt[0]/1000.0 < 25: continue
    if ds.lep_etcone20[0]/ds.lep_pt[0] > 0.15: continue
    if ds.lep_ptcone30[0]/ds.lep_pt[0] > 0.15: continue
        
    if ds.lep_pt[1]/1000.0 < 25: continue
    if ds.lep_etcone20[1]/ds.lep_pt[1] > 0.15: continue
    if ds.lep_ptcone30[1]/ds.lep_pt[1] > 0.15: continue
    
    #New cuts
    ## Require loose criteria.
    if ds.lep_type[0] == 11:
        if (abs(ds.lep_eta[0]) > 2.47 or abs(ds.lep_eta[1]) > 2.47 ) : continue
        if abs(ds.lep_trackd0pvunbiased[0] / ds.lep_tracksigd0pvunbiased[0]) > 5: continue
        if abs(ds.lep_trackd0pvunbiased[1] / ds.lep_tracksigd0pvunbiased[1]) > 5: continue
    if ds.lep_type[0] == 13:
        if (abs(ds.lep_eta[0]) > 2.7 or abs(ds.lep_eta[1]) > 2.7 ) : continue
        if abs(ds.lep_trackd0pvunbiased[0] / ds.lep_tracksigd0pvunbiased[0]) > 3: continue
        if abs(ds.lep_trackd0pvunbiased[1] / ds.lep_tracksigd0pvunbiased[1]) > 3: continue
    
    
    ## Set Lorentz vectors: 
    l1.SetPtEtaPhiE(ds.lep_pt[0]/1000., ds.lep_eta[0], ds.lep_phi[0], ds.lep_E[0]/1000.);
    l2.SetPtEtaPhiE(ds.lep_pt[1]/1000., ds.lep_eta[1], ds.lep_phi[1], ds.lep_E[1]/1000.);
    
    #New cuts
    if abs(ds.lep_z0[0] * sin(l1.Theta())) > 0.5: continue 
    if abs(ds.lep_z0[1] * sin(l2.Theta())) > 0.5: continue
    
    ## Variables are stored in the TTree with unit MeV, so we need to divide by 1000 
    ## to get GeV, which is a more practical and commonly used unit. 
    dileptons = l1 + l2;   
    if dileptons.M()<100:continue

    columns["lep_pt1"].append(ds.lep_pt[0]/1000.0)
    columns["lep_eta1"].append(ds.lep_eta[0])
    columns["lep_phi1"].append(ds.lep_phi[0])
    columns["lep_E1"].append(ds.lep_E[0]/1000.0)
    columns["lep_m1"].append(l1.M())
    
    columns["lep_pt2"].append(ds.lep_pt[1]/1000.0)
    columns["lep_eta2"].append(ds.lep_eta[1])
    columns["lep_phi2"].append(ds.lep_phi[1])
    columns["lep_E2"].append(ds.lep_E[1]/1000.0)
    columns["lep_m2"].append(l2.M())
    
    columns["met"].append(ds.met_et/1000.0) 
    
    if ds.channelNumber == 392918:
        columns["label"].append(1)
    else:
        columns["label"].append(0)
    if isData == 0:
        weight = ((ds.mcWeight)*(ds.scaleFactor_PILEUP)*
                 (1)*(1)*
                 (1))*((ds.XSection*lumi)/ds.SumWeights)
        columns["weight"].append(weight)
    else:
        columns["weight"].append(-999.0)
        


        
print("Done!")
df = pd.DataFrame(data=columns)
df.info

Total events 100000/21867152
Total events 200000/21867152
Total events 300000/21867152
Total events 400000/21867152
Total events 500000/21867152
Total events 600000/21867152
Total events 700000/21867152
Total events 800000/21867152
Total events 900000/21867152
Total events 1000000/21867152
Total events 1100000/21867152
Total events 1200000/21867152
Total events 1300000/21867152
Total events 1400000/21867152
Total events 1500000/21867152
Total events 1600000/21867152
Total events 1700000/21867152
Total events 1800000/21867152
Total events 1900000/21867152
Total events 2000000/21867152
Total events 2100000/21867152
Total events 2200000/21867152
Total events 2300000/21867152
Total events 2400000/21867152
Total events 2500000/21867152
Total events 2600000/21867152
Total events 2700000/21867152
Total events 2800000/21867152
Total events 2900000/21867152
Total events 3000000/21867152
Total events 3100000/21867152
Total events 3200000/21867152
Total events 3300000/21867152
Total events 340000

<bound method DataFrame.info of            lep_pt1  lep_eta1  lep_phi1       lep_E1    lep_m1     lep_pt2  \
0       617.628063 -0.456145  0.651721   683.004312 -0.112884  149.135312   
1        71.277578 -0.980773  1.997837   108.396687 -0.027978   63.859563   
2       624.291125 -1.800602 -3.065241  1941.074625  0.331861   35.573598   
3       384.536656 -1.684730 -2.843123  1072.182250 -0.255588   39.422805   
4       416.827063 -1.094879 -0.861139   692.641500 -0.134606   42.473062   
...            ...       ...       ...          ...       ...         ...   
283337  365.936906 -0.683010 -1.241499   454.662281  0.051307   60.022266   
283338  634.140750  0.904956 -0.712485   912.015688  0.226749  521.212156   
283339  245.439516  0.785956  2.223517   325.230406  0.042896  165.660297   
283340  226.355875 -0.878349  1.779889   319.432469  0.072644  137.475063   
283341  751.459250  0.168067  1.624177   762.097250  0.040237  644.191000   

        lep_eta2  lep_phi2      lep_E2    l

In [21]:
df.to_csv("test.csv")

## 3. Scale and classify the histograms (MC only) 

Before we are ready to make plots we need to do some further processing of the histograms we made above. The information necessary for doing the two steps below is found in the file **Infofile.txt**.   
1. We need to **scale** the histograms to the right cross section and luminosity. Why? When making the MC samples a certain number of events is simulated, which will usually not correspond to the number of events in our data. The expected number of events from a certain kind of process is given by $N=\sigma L$, where $\sigma$ is the cross section and $L$ is the integrated luminosity. Therefore we need to scale each histogram by a scale factor <br> <br>
$$sf = \frac{N}{N_{MC}} = \frac{ \sigma L }{N_{MC}},$$ <br>  where $N_{MC}$ is the number of generated MC events.  <br> <br>
2. We also need to **classify** the background processes into different categories. This is necessary when we eventually want to make the characteristic colorful background plots you might have seen before.  

### 3.1 Make new histograms 
Maybe a bit depressingly we have to make a set of new histograms, this time corresponding to the different background categories, instead of the dataset IDs. Notice that these new histograms are made in a very similar way as above, i.e. with the same range and binning. 

In [ ]:
H_mll = {}; H_lep_pt = {}; H_met = {}; 

In [ ]:
L = 10.06

In [ ]:
for i in Backgrounds: 
    H_mll[i] = R.TH1F() 
    H_lep_pt[i] = R.TH1F() 
    H_met[i] = R.TH1F() 

In [ ]:
for i in Backgrounds: 
    H_mll[i].SetNameTitle("hist_mll", "Invariant mass"); 
    H_lep_pt[i].SetNameTitle("hist_lep_pt", "Lepton pT"); 
    H_met[i].SetNameTitle("hist_met", "Missing ET");
    H_mll[i].SetBins(20,0,500); 
    H_lep_pt[i].SetBins(20,0,1000);
    H_met[i].SetBins(20,0,500); 

### 3.2 Scale and add histograms 
Now we read our info file, scale all (old) histograms, and then add them to the new histograms we just defined.  

In [ ]:
for i in Backgrounds: 
    H_mll[i].Reset(); 
    H_lep_pt[i].Reset(); 
    H_met[i].Reset();

In [ ]:
for dsid in hist_mll.keys(): 
    
    Type = MCcat[dsid]
    
    H_mll[Type].Add(hist_mll[dsid]); 
    H_lep_pt[Type].Add(hist_lep_pt[dsid]); 
    H_met[Type].Add(hist_met[dsid]); 
    
infofile.close()

### 3.3 Color the histograms 
Make yet another map, this time containing the colors you want the backgrounds to have, and then set the colors of your histograms. Note that colors are defined by integers in ROOT. If you are not happy with the colors chosen below you can have look at the [TColor](https://root.cern.ch/doc/master/classTColor.html) class reference for more options. 

In [ ]:
colors = {}

In [ ]:
colors["Diboson"] = R.kGreen; 
colors["Zjets"] = R.kYellow; 
colors["ttbar"] = R.kRed;
colors["singleTop"] = R.kBlue-7; 
colors["Wjets"] = R.kBlue+3; 
colors["topX"] = R.kOrange+1; 
colors["Higgs"] = R.kMagenta; 
colors["Wjetsincl"] = R.kBlue-10;
colors["Zjetsincl"] = R.kYellow-9;

In [ ]:
for h in Backgrounds: 
    H_mll[h].SetFillColor(colors[h]); 
    H_met[h].SetFillColor(colors[h]);
    H_lep_pt[h].SetFillColor(colors[h]);
    
    H_mll[h].SetLineColor(colors[h]); 
    H_met[h].SetLineColor(colors[h]);
    H_lep_pt[h].SetLineColor(colors[h]);

## 4. Stack and plot the histograms

Finally we have arrived to the part where we can plot the results of all the work done above. For each variable we need to stack the backgrounds on top of each other, which is done by using the [THStack](https://root.cern.ch/doc/master/classTHStack.html) class. In the example below we do this for two variables; invariant mass and missing $E_T$.   

In [ ]:
stack_mll = R.THStack("Invariant mass", "");
stack_met = R.THStack("Missing ET", ""); 
stack_lep_pt = R.THStack("Lepton pT", ""); 

In [ ]:
for h in Backgrounds: 
    stack_mll.RecursiveRemove(H_mll[h]); ## Remove previously stacked histograms  
    stack_met.RecursiveRemove(H_met[h]);
    stack_lep_pt.RecursiveRemove(H_lep_pt[h]);
    stack_mll.Add(H_mll[h]); 
    stack_met.Add(H_met[h]);
    stack_lep_pt.Add(H_lep_pt[h]); 

Now we make a legend (see [TLegend](https://root.cern.ch/doc/master/classTLegend.html)), and add  the different backgrounds. Next we make a canvas (see [TCanvas](https://root.cern.ch/doc/master/classTCanvas.html)), which is allways necessary when we want to make a plot. Then you draw the stack and the legend, and display them by drawing the canvas. We can also specify axis labels and a bunch of other stuff. 

In [ ]:
R.gStyle.SetLegendBorderSize(0); ## Remove (default) border around legend 
leg = R.TLegend(0.65, 0.60, 0.9, 0.85); 

In [ ]:
leg.Clear();
for i in Backgrounds: 
    leg.AddEntry(H_mll[i], i, "f")  ## Add your histograms to the legend
leg.AddEntry(hist_mll_d, "Data", "lep") 

In [ ]:
C = R.TCanvas("c", "c", 600, 600)

In [ ]:
R.gPad.SetLogy() ## Set logarithmic y-axis

In [ ]:
hist_mll_d.SetLineColor(R.kBlack); 
hist_mll_d.SetMarkerStyle(R.kFullCircle); 
hist_mll_d.SetMarkerColor(R.kBlack); 

In [ ]:
stack_mll.Draw("hist"); 
stack_mll.SetMaximum(1E6); 
stack_mll.SetMinimum(1); 
stack_mll.GetYaxis().SetTitle("# events");
stack_mll.GetYaxis().SetTitleOffset(1.3); 
stack_mll.GetXaxis().SetTitle("m_{ll} (GeV)");
stack_mll.GetXaxis().SetTitleOffset(1.3);
hist_mll_d.Draw("same E"); 
leg.Draw();
C.Draw();

In [ ]:
hist_met_d.SetLineColor(R.kBlack); 
hist_met_d.SetMarkerStyle(R.kFullCircle); 
hist_met_d.SetMarkerColor(R.kBlack); 

In [ ]:
stack_met.Draw("hist"); 
stack_met.SetMaximum(1E6); 
stack_met.GetYaxis().SetTitle("# events");
stack_met.GetYaxis().SetTitleOffset(1.3); 
stack_met.GetXaxis().SetTitle("E_{T}^{miss} (GeV)");
stack_met.GetXaxis().SetTitleOffset(1.3);
hist_met_d.Draw("same e"); 
leg.Draw();
C.Draw(); 

In [ ]:
hist_lep_pt_d.SetLineColor(R.kBlack); 
hist_lep_pt_d.SetMarkerStyle(R.kFullCircle); 
hist_lep_pt_d.SetMarkerColor(R.kBlack); 

In [ ]:
stack_lep_pt.Draw("hist"); 
stack_lep_pt.SetMaximum(1E6); 
stack_lep_pt.GetYaxis().SetTitle("# events");
stack_lep_pt.GetYaxis().SetTitleOffset(1.3); 
stack_lep_pt.GetXaxis().SetTitle("p_{T} (GeV)");
stack_lep_pt.GetXaxis().SetTitleOffset(1.3);
hist_lep_pt_d.Draw("same e"); 
leg.Draw();
C.Draw(); 